# Exercise 1

Loading the data from the arff file and converting it into a dataframe

In [ ]:
from scipy.io.arff import loadarff
import pandas as pd

# Load the data
data = loadarff('./column_diagnosis.arff')
df = pd.DataFrame(data[0])
df['class'] = df['class'].str.decode('utf-8')
df.head()

Separating the input data and the output data, required for some sklean functions

In [ ]:
x = df.drop('class', axis=1)
y = df['class']

Checking the discriminative power of each feature in accordance with f_classif criterion. The higher the value, the more discriminative the feature is.

In [ ]:
from sklearn.feature_selection import f_classif

fimportance = f_classif(x, y)

scores = fimportance[0]

highest_discriminative_power = x.columns.values[scores.argmax()]
lowest_discriminative_power = x.columns.values[scores.argmin()]
print('Input variable with highest discriminative power: ', highest_discriminative_power)
print('Input variable with lowest discriminative power: ', lowest_discriminative_power)

Plotting the class-conditional probability density functions of these two input variables.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot the class-conditional probability density functions of these two input variables.

sns.displot(data=df, x=highest_discriminative_power, kind='kde', hue='class', fill=True)
plt.title('Class-conditional probability density function for ' + highest_discriminative_power)
plt.show()

sns.displot(data=df, x=lowest_discriminative_power, kind='kde', hue='class', fill=True)
plt.title('Class-conditional probability density function for ' + lowest_discriminative_power)
plt.show()

# Exercise 2

First, we will split the data into training and testing sets. We will use 70% of the data for training and 30% for testing.

In [ ]:
from sklearn import metrics, tree
from sklearn.model_selection import train_test_split
import numpy as np

# Split the data into training and test sets
avg_train_accs, avg_test_accs = [], []
# Define the depth limits
depth_limits = [1,2,3,4,5,6,8,10]
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=0, stratify=y)

Now, we will use the training data to train the model. We will use the DecisionTreeClassifier from sklearn.tree to train the model. We will be doing 10 runs of the model and will be averaging the accuracy of the model.

In [ ]:
n_runs = 10
for i in depth_limits:
    train_accs, test_accs = [], []
    for j in range(n_runs):
        # train classifier    
        predictor = tree.DecisionTreeClassifier(max_depth=i, random_state=0)
        # fit classifier
        predictor.fit(X_train, y_train)
        # test classifier
        y_pred_test = predictor.predict(X_test)
        y_pred_train = predictor.predict(X_train)
        
        # calculate accuracy
        train_acc = round(metrics.accuracy_score(y_train, y_pred_train),2)        
        test_acc = round(metrics.accuracy_score(y_test, y_pred_test),2)
        
        train_accs.append(train_acc)
        test_accs.append(test_acc)
    
    avg_train_accs.append(np.mean(train_accs))
    avg_test_accs.append(np.mean(test_accs))

Assess in a single plot both the training and testing accuracies of a decision tree with depth limits 

In [ ]:
sns.lineplot(x=depth_limits, y=avg_train_accs, label='Training Accuracy')
sns.lineplot(x=depth_limits, y=avg_test_accs, label='Testing Accuracy')
sns.scatterplot(x=depth_limits, y=avg_train_accs)
sns.scatterplot(x=depth_limits, y=avg_test_accs)
plt.grid()
plt.xlabel('Depth Limit')
plt.ylabel('Accuracy')
plt.title('Training and Testing Accuracy for Decision Tree')
plt.show()

# Exercise 3

In the results from Question 2, we can observe the relationship between the max depth of the decision tree and its training and testing accuracies. We can see some key points from the plot:
- As the max depth of the decision tree increases, the training accuracy generally improves. This is expected because a deeper tree can fit the training data better, and hence, the training accuracy will improve.
  
- When analyzing the testing accuracy, it shows a different trend. Initially, as the tree uses a smaller depth limits, the testing accuracy improves. But, after a certain point (around depth of 5), the testing accuracy starts to decline. This is a clear sign of overfitting. The model is overfitting the training data and hence, the testing accuracy is declining.
  
- Considering the prior analysis, the optimal max depth for the decision tree is 5. Beyond that point, the model tens to overfit the training data and hence, the testing accuracy starts to decline.

# Exercise 4

In [ ]:
# Create a Decision Tree Classifier with the specified parameters
predictor = tree.DecisionTreeClassifier(random_state=0, min_samples_leaf=20)

# Fit the classifier on all available data
predictor.fit(x, y)

# Plot the decision tree
plt.figure(figsize=(12, 8))
tree.plot_tree(predictor, filled=True, feature_names=x.columns.values, class_names=y, rounded=True)
plt.title("Decision Tree Classifier")
plt.show()

## 4.2
